In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
import numpy as np
import json
import os

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m4.xlarge',
                          framework_version='2.4.1', 
                          sagemaker_session=sagemaker_session,
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={ 'epochs': 2 },
                         )

In [6]:
tf_estimator.fit({'train': 's3://birdsound-identification-dataset/data/'})

2021-07-16 21:39:26 Starting - Starting the training job...
2021-07-16 21:39:49 Starting - Launching requested ML instancesProfilerReport-1626471566: InProgress
......
2021-07-16 21:40:49 Starting - Preparing the instances for training......
2021-07-16 21:41:56 Downloading - Downloading input data...
2021-07-16 21:42:10 Training - Downloading the training image..2021-07-16 21:42:31.569128: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-07-16 21:42:31.573699: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-07-16 21:42:31.687040: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-07-16 21:42:35,282 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-07-16 21:42:35,289 sagemaker-training-

In [7]:
predictor = tf_estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------------!

In [2]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data='s3://sagemaker-us-east-1-495733045912/tensorflow-training-2021-07-16-21-39-25-994/output/model.tar.gz',framework_version='2.4.1',  role=role)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!